# 「シゴトがはかどる Python 自動処理の教科書」 クジラ飛行机 著 (つづき)


## メールや LINE／SNS を自動化しよう


**メールを自動送信(Gmail,Yahoo!メール,Outlook.com)**


In [ ]:
# my_gmail_account.py
account = "???????????@gmail.com"
password = "??????????"

In [ ]:
import smtplib, ssl
from email.mime.text import MIMEText

# import my_gmail_account as gmail


# メイン処理
def send_test_email():
    msg = make_mime_text(
        mail_to=account, subject="メールの送信テスト", body="こんにちは。届きましたか？テスト。テスト。"
    )
    send_gmail(msg)


# メールデータ(MIME)を生成
def make_mime_text(mail_to, subject, body):
    msg = MIMEText(body, "html")
    msg["subject"] = subject
    msg["To"] = mail_to
    msg["from"] = account
    return msg


# Gmailに接続
def send_gmail(msg):
    server = smtplib.SMTP_SSL(
        "smtp.gmail.com", 465, context=ssl.create_default_context()
    )
    server.set_debuglevel(0)
    server.login(account, password)
    server.send_message(msg)


if __name__ == "__main__":
    send_test_email()
    print("ok.")

**Outlook.com の場合**


In [14]:
import smtplib
from email.mime.text import MIMEText

account = "??????????@outlook.com"
password = "??????????"


# メイン処理
def send_test_email():
    msg = make_mime_text(
        mail_to=account, subject="メールの送信テスト", body="こんにちは。届きましたか？テスト。テスト。"
    )
    send_outlookmail(msg)


# メールデータ(MIME)を生成
def make_mime_text(mail_to, subject, body):
    msg = MIMEText(body, "plain")
    msg["subject"] = subject
    msg["To"] = mail_to
    msg["from"] = account
    return msg


# Outlookのサーバーに接続
def send_outlookmail(msg):
    server = smtplib.SMTP("smtp.office365.com", 587)
    server.ehlo()
    server.starttls()
    server.ehlo()
    server.login(account, password)
    server.send_message(msg)


if __name__ == "__main__":
    send_test_email()
    print("ok.")

ok.


**LINE を自動送信しよう**


In [17]:
import requests

acc_token = "????????????????????????"


def send_line(msg):
    url = "https://notify-api.line.me/api/notify"
    headers = {"Authorization": "Bearer " + acc_token}
    payload = {"message": msg}
    requests.post(url, headers=headers, params=payload)


if __name__ == "__main__":
    send_line("Pythonからこんにちは")
    print("ok!")

ok!


**タスクスケジューラで定期処理を自動実行**


In [21]:
# get_tenki.py
import requests

save_file = "chap5/天気.txt"
api_url = "https://api.aoikujira.com/tenki/week.php?fmt=ini&city=319"
tenki = requests.get(api_url).text

with open(save_file, "wt", encoding="utf-8") as f:
    f.write(tenki)

バッチファイルを作って登録する方法


## 業務で役立つ自動化テクニック集


**社内で使える簡易 Web サーバー**


In [ ]:
from flask import Flask

app = Flask(__name__)  # flaskオブジェクトを生成


# / へアクセスがあった場合の処理
@app.route("/")
def roote():
    return "Hello!"


# /testへアクセスがあった場合の処理
@app.route("/test")
def test():
    return "Test..."


if __name__ == "__main__":
    app.run(debug=True, host="0.0.0.0", port=5000)

In [ ]:
# share.py
from flask import Flask
import glob

app = Flask(__name__)


@app.route("/")
def roote():
    files = glob.glob("static/*")
    html = '<html><meta charset="utf-8"><body>'
    html += "<h1>ファイル一覧</h1>"
    for f in files:
        html += '<P><a href="{0}">{0}</a></p>'.format(f)
    html += "</body></html>"
    return html


if __name__ == "__main__":
    app.run(debug=True, host="0.0.0.0", port=5000)

簡単な正規表現で基本に慣れよう

In [1]:
import re

target = r'app dog art pot'
pattern = r'a..'
a = re.findall(pattern, target)
print(a)

['app', 'art']


英和辞書から正規表現で英単語を抽出しよう

In [2]:
import re

dict_file = './ejcict-hand-utf8.txt'

while True:
    print('正規表現を入力してください。[q]で終了。')
    re_str = input()
    if re_str == '':
        continue
    if re_str == 'q':
        break
    with open(dict_file, 'rt', encoding='utf-8') as f:
        cont = 0
        while True:
            line = f.readline() #1行だけ読む
            if not line :
                break
            word, mean = line.split('\t')
            if re.search(re_str, word):
                print(word)
                cnt += 1
                if cnt > 50: #最大50件
                    break

正規表現を入力してください。[q]で終了。


正規表現を用いて郵便番号を取り出してみよう

In [2]:
import re

with open('chap6/address.txt', 'rt', encoding='utf-8') as f:
    text = f.read()
    a = re.findall(r'〒\d{3}-\d{4}',text)
    print(a)

['〒231-0012', '〒120-0015']


In [3]:
import re

with open('chap6/address.txt', 'rt', encoding='utf-8') as f:
    text = f.read()
    pattern = r'〒(\d{3})-(\d{4})'
    for m in re.finditer(pattern, text):
        print('---')
        print('場所:', m.span())
        print('全体:', m.group(0))
        print('前半:', m.group(1))
        print('後半:', m.group(2))

---
場所: (0, 9)
全体: 〒231-0012
前半: 231
後半: 0012
---
場所: (35, 44)
全体: 〒120-0015
前半: 120
後半: 0015


電話番号形式を変更する --置換

In [19]:
import re
# 対象文字列
tel = r'[電話]000-1111-2222'
# 検索パターン
pattern = r'(\d+)-(\d+)-(\d+)'
# 置換後
rep = r'(\1) \2 - \3'
print(re.sub(pattern, rep, tel))

[電話](000) 1111 - 2222


**マウス・キーボードの自動化**

PyAutoGUIのインストール

In [21]:
pip install pyautogui


     ---------------------------------------- 0.0/61.2 kB ? eta -:--:--
     ------------ ------------------------- 20.5/61.2 kB 640.0 kB/s eta 0:00:01
     ------------------------------- ------ 51.2/61.2 kB 650.2 kB/s eta 0:00:01
     -------------------------------------- 61.2/61.2 kB 463.6 kB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     ----------------------

In [22]:
pip list

Package                   VersionNote: you may need to restart the kernel to use updated packages.

------------------------- ------------
altgraph                  0.17.4
asttokens                 2.4.0
backcall                  0.2.0
beautifulsoup4            4.12.2
brotlipy                  0.7.0
certifi                   2023.7.22
cffi                      1.15.1
charset-normalizer        2.0.4
click                     8.0.4
colorama                  0.4.6
comm                      0.1.4
cryptography              41.0.3
debugpy                   1.8.0
decorator                 5.1.1
EasyProcess               1.1
entrypoint2               1.1
et-xmlfile                1.1.0
executing                 2.0.0
Flask                     2.2.2
html5lib                  1.1
idna                      3.4
ipykernel                 6.25.2
ipython                   8.16.1
itsdangerous              2.0.1
jedi                      0.19.1
Jinja2                    3.1.2
jupyter_client            

In [24]:
pip install opencv_python

     ---------------------------------------- 0.0/61.2 kB ? eta -:--:--
     -------------------- ------------------- 30.7/61.2 kB 1.4 MB/s eta 0:00:01
     ---------------------------------------- 61.2/61.2 kB 1.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/38.1 MB ? eta -:--:--
   ---------------------------------------- 0.2/38.1 MB 6.3 MB/s eta 0:00:07
    --------------------------------------- 0.5/38.1 MB 6.8 MB/s eta 0:00:06
   - -------------------------------------- 1.1/38.1 MB 8.4 MB/s eta 0:00:05
   - -------------------------------------- 1.7/38.1 MB 9.6 MB/s eta 0:00:04
   -- ------------------------------------- 2.2/38.1 MB 9.9 MB/s eta 0:00:04
   -- ------------------------------------- 2.6/38.1 MB 9.7 MB/s eta 0:00:04
   --- ------------------------------------ 3.3/38.1 MB 10.6 MB/s eta 0:00:04
   ---- ----------------------------------- 3.9/38.1 MB 10.7 MB/s eta 0:00:04
   ---- ----------------------------------- 4.4/38.1 MB 10.9 MB/s eta 0:00:04
  

windowsのメモ帳を起動して、Hello,Python!と文字を書き込むプログラム

In [35]:
import time, subprocess, platform
import pyautogui as pa

message = 'Hello,Python!'
if platform.system() == 'Windows':
    subprocess.Popen(r'c:\Windows\notepad.exe')
    time.sleep(10)

pa.write(message)